In [3]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics.pairwise import cosine_similarity

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
face_recognition_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
face_recognition_model.trainable = False

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    return img_array

def detect_faces_from_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))


        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, 'Face Detected!', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        cv2.imshow('Detected Faces in Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def recognize_faces(target_image_path, database_images):
    target_image = preprocess_image(target_image_path)
    target_embedding = face_recognition_model.predict(target_image)
    similarities = []
    for db_image_path in database_images:
        db_image = preprocess_image(db_image_path)
        db_embedding = face_recognition_model.predict(db_image)
        similarity = cosine_similarity(target_embedding.reshape(1, -1), db_embedding.reshape(1, -1))[0][0]
        similarities.append(similarity)
    return similarities

video_path = '/content/h.12.13 PM.mov'
detect_faces_from_video(video_path)

target_image_path = '/content/Unknown-1.jpeg'
database_images = ['/content/Unknown-1.jpeg', '/content/Unknown-2 7.57.34 PM.jpeg']

similarities = recognize_faces(target_image_path, database_images)

for i, similarity in enumerate(similarities):
    print(f"Similarity with face {i+1}: {similarity}")


1/1 [==============================] - 0s 66ms/step
Similarity with face 1: 0.9999998807907104
Similarity with face 2: 0.18631242215633392
